In [2]:
import sys


import numpy as np


import torch
import torch.distributed as dist


from importlib import reload
from torch.utils.data import DataLoader

from matplotlib import pyplot as plt
import numpy as np

from sklearn.linear_model import  RidgeCV

from regressors import stats    
import utility

reload(utility)
from statsmodels.stats.multitest import multipletests
import statsmodels.api as sm

from sklearn.decomposition import PCA


In [3]:
image_gene_dataset = utility.image_gene_dataset(train = True, cv = None,)
sid, image_features, genes = image_gene_dataset[:]

/ocean/projects/asc170022p/juc91/anaconda/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [13]:
image_gene_dataset = utility.image_gene_dataset(train = True, cv = None,)
sid, image_features, genes = image_gene_dataset[:]

X_raw = image_features.view(-1, 581 * 128).cpu().data.numpy()
M_PCA = PCA(n_components = 128)
X = M_PCA.fit_transform(X_raw)

genes = genes.numpy()

/ocean/projects/asc170022p/juc91/anaconda/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [2]:
# image_gene_dataset = utility.image_gene_dataset(train = True, cv = None,)
# sid, image_features, genes = image_gene_dataset[:]

# X = image_features.view(-1, 581 * 128).numpy()
# genes = genes.numpy()


/ocean/projects/asc170022p/juc91/anaconda/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
p_values = []


for gene_idx in range( genes.shape[1] ):
    y = genes[:, gene_idx]
    M_OLS = sm.OLS(y, X).fit()
    p_values.append(M_OLS.f_pvalue)

    
_, adj_p, _, _ = multipletests(p_values)

In [15]:
# Get list of genes with different thresholds
p_thresholds = [1., .05, .01, 5e-3, 1e-3, 5e-4, 1e-4, 5e-5, 1e-5 ] 

for ppp in p_thresholds:
    
    f = open("gene_list_PCA/THOLD_{}.txt".format(ppp), "w")
    f.write("index, gene name\n")
    for iii in np.where( adj_p <= ppp )[0]:
        f.write("{},{}\n".format(iii, image_gene_dataset.genelist[iii]))
    f.close()
    